In [22]:
from random import randint

In [23]:
class Node:
    def __init__(self, key):
        self.key = key
        self.edges = None
        self.next = None
        self.color = 'white' 
        
    def __repr__(self):
        return f'Node "{self.key}"'
    
    def __iter__(self):
        item = self.edges
        while item is not None:
            yield item
            item = item.next
    
        
class Edge:
    def __init__(self, node_in, weight=None, key=None):
        self.next = None
        self.node_in = node_in
        self.weight = weight
        self.key = key
        
    def __repr__(self):
        return f'Edge "key-{self.key}, weight-{self.weight}, to-{self.node_in}"'
        
class Graph:
    def __init__(self, node=None, adges=Node):
        self.head_node = node
        
    def __iter__(self):
        item = self.head_node
        while item is not None:
            yield item
            item = item.next
            
    def add_node(self, node):
        if self.head_node is None:
            self.head_node = node
            return
        last_node = self.head_node
        while last_node.next:
            last_node = last_node.next
        last_node.next = node
    
    
    def search_node(self, key):
        if self.head_node is None:
            return
        next_node = self.head_node
        if next_node.key == key:
            return next_node
        while next_node.next:
            next_node = next_node.next
            if next_node.key == key:
                return next_node
    
    
    def add_edge(self, node_key_out, node_key_in,  weight=None, key=None):
        node_out = self.search_node(node_key_out)
        node_in = self.search_node(node_key_in)
        edge = Edge(node_in, weight, key)
        if not node_out or not node_in:
            print('Не найдена вершина')
            return
        if node_out.edges is None:
            node_out.edges = edge
            return
        next_edge = node_out.edges
        while next_edge.next:
            next_edge = next_edge.next
        next_edge.next = edge
        
    def remove_edge_out(self, node_key_out, node_key_in):
        node_out = self.search_node(node_key_out)
#         node_in = self.search_node(node_key_in)
        if not node_out.edges:
            return
        next_edge = node_out.edges
        if next_edge.node_in.key == node_key_in:
            node_out.edges = next_edge.next
            return
        while next_edge.next:
            if next_edge.next.node_in.key == node_key_in:
                next_edge.next = next_edge.next.next
                return
            next_edge = next_edge.next
            
    def remove_node(self, node_key):
        self.remove_edges_in(node_key)
        
        next_node = self.head_node
        if next_node.key == node_key:
            self.head_node = next_node.next
            return
        while next_node:
            if next_node.next.key == node_key:
                next_node.next = next_node.next.next
                return
            next_node = next_node.next
        
        
    def del_edges(self, node, node_key_in):
        if not node.edges:
            return
        edge = node.edges
        if edge.node_in.key == node_key_in:
            node.edges = edge.next
        while edge.next:
            if edge.next.node_in.key == node_key_in:
                edge.next = edge.next.next
                continue
            edge = edge.next
    
    def remove_edges_in(self, node_key_in):
        if not self.head_node:
            return       
        node = self.head_node
        while node:
            self.del_edges(node, node_key_in)            
            node = node.next
            
    def get_adjacent_nodes(self, node):
        return [item.node_in for item in node]
    
    def refresh_color(self):
        for item in self:
            item.color = 'white'
            
    def nodes_list(self):
        return [item.key for item in self]
    


In [24]:
def DFS(graph, node, visited):
    if node.color != 'white':
        return
# проверка и действие
    node.color = 'gray'
    adjacent_nodes = graph.get_adjacent_nodes(node)
    for item in adjacent_nodes:
        if item.color == 'white':
            # проверка и действие
            DFS(graph, item, visited)
    node.color = 'black'
    visited.append(node.key)

In [25]:
# первый граф из лекции
g = Graph()

g.add_node(Node('A'))
g.add_node(Node('B'))
g.add_node(Node('C'))
g.add_node(Node('D'))
g.add_node(Node('E'))
g.add_node(Node('F'))
g.add_node(Node('G'))
g.add_node(Node('H'))


# прямой граф
g.add_edge('E','A')
g.add_edge('A','B')
g.add_edge('B','C')
g.add_edge('D','C')
g.add_edge('C','D')
g.add_edge('H','D')
g.add_edge('B','E')
g.add_edge('E','F')
g.add_edge('B','F')
g.add_edge('G','F')
g.add_edge('F','G')
g.add_edge('C','G')
g.add_edge('H','G')
g.add_edge('D','H')

In [26]:
def reves_edges_in_graph(graph):
    new_graph = Graph()
    revers_edges = []
    for item_node in graph:
        new_graph.add_node(Node(item_node.key))
        for item_edge in item_node:
            revers_edges.append((item_edge.node_in.key, item_node.key))
    for item in revers_edges:
        new_graph.add_edge(item[0], item[1])
    return new_graph

In [27]:
# DFS с рекурсией
def kosaraju(g):
    new_graph = reves_edges_in_graph(g)
    nodes_key_list = new_graph.nodes_list()
    visited = []
    result = []
    list_for_search = nodes_key_list.copy()
    len_nodes_key_list = len(nodes_key_list)
    while len(result) < len_nodes_key_list:
        random_key = list_for_search[randint(0, len(list_for_search) - 1)]
        node = new_graph.search_node(random_key)
        DFS(new_graph, node, visited)
        temp = visited[len(result):].copy()
#         temp.reverse()
        result.extend(temp)
        list_for_search = list(set(nodes_key_list) - set(result)) 
        
    visited = []
    answer = []
    temp = []
    while result:
        node_key = result.pop()
        if node_key in visited:
            continue
        node = g.search_node(node_key)
        DFS(g, node, visited)
        answer.append(list(set(visited) - set(temp)))
        temp = visited.copy()
    g.refresh_color()
    return answer

In [28]:
kosaraju(g)

[['G', 'F'], ['D', 'C', 'H'], ['A', 'E', 'B']]

In [29]:
for _ in range(1000):
    r = kosaraju(g)
    if len(r) != 3:
        print('Error  ', r)
        break

In [30]:
# второй граф из лекции
g = Graph()

g.add_node(Node('A'))
g.add_node(Node('B'))
g.add_node(Node('C'))
g.add_node(Node('D'))
g.add_node(Node('E'))
g.add_node(Node('F'))
g.add_node(Node('G'))
g.add_node(Node('H'))

g.add_edge('A','E')
g.add_edge('A','F')
g.add_edge('A','G')
g.add_edge('B','A')
g.add_edge('B','D')
g.add_edge('B','F')
g.add_edge('C','G')
g.add_edge('C','H')
g.add_edge('H','G')
g.add_edge('H','D')

In [31]:
kosaraju(g) 

[['D'], ['F'], ['E'], ['G'], ['A'], ['B'], ['H'], ['C']]

In [32]:
for _ in range(1000):
    r = kosaraju(g)
    if len(r) != 8:
        print('Error  ', r)
        break